In [4]:
import random
from collections import defaultdict
import time

In [14]:
def graph_counterexample(n):
    assert n % 2 == 1, f"number of edges: ({n = }) must be odd"
    
    edges = [(0, 1, 0)]
    weights = [float('inf')]
    j = 1
    for i in range(2, n//2+2):
        edges.append((0, i, j))
        edges.append((1, i, j+1))
        weights += [2, 1]
        j += 2
    G = Graph(edges)
#     G.plot()
#     G.show()
    return Matroid(G), weights

In [6]:
# https://www.sciencedirect.com/science/article/pii/0012365X75900758
def knuth_random_matroid(n):
    universe = frozenset(range(n))
    r = 0
    F = [set(frozenset())]
    while universe not in F[r]:
        # generate covers
        F_next = set(frozenset())
        if r == 0:
            for a in universe:
                F_next.add(frozenset([a]))
        else:
            for A in F[r]:
                for a in universe - A:
                    x = set(A)
                    x.add(a)
                    F_next.add(frozenset(x))

        #enlarge
        if r != 0:
            # adjust the number of times this loop is performed to generate different matroids
            for _ in range(random.randint(0, n//2)):
                A = set(random.choice(list(F_next)))
                F_next.remove(A)
                if len(list(universe-A)) != 0:
                    A.add(random.choice(list(universe-A)))
                F_next.add(frozenset(A))

        #test with n = 10 for example in paper
#         if r == 1:
#             pi = [[1,3,4],[1,5,9],[2,5,6],[3,5,8],[3,7,9],[2,3,8]]
#             for x in pi:
#                 F_next.add(frozenset(x))

        # superpose
        isContained = False
        while not isContained:
            isContained = True
            not_contained = []
            for A in F_next:
                for B in F_next:
                    if A == B: continue
                    if not contained(A.intersection(B), F[r]): 
                        not_contained.append((A, B))
                        isContained = False
            for AB in not_contained:
                A, B = AB
                if A in F_next and B in F_next:
                    F_next.remove(A)
                    F_next.remove(B)
                    F_next.add(A.union(B))
        F.append(F_next)
        r += 1
    
    closed_sets = []
    for rank, X in enumerate(F):
        for Y in X:
            closed_sets.append((rank, Y))

    return Matroid(universe, circuit_closures = closed_sets)

def contained(AandB, Fr):
    if len(AandB) == 0: return True
    for C in Fr:
        if AandB.issubset(C): return True
    False

In [46]:
# Import any algorithm implementations here
class FreeOrderMatroidAlgo:
    def __init__(self, matroid, weights):
        assert matroid.size() == len(weights)

        self.matroid = matroid
        self.weights = weights

class Conjecture(FreeOrderMatroidAlgo):
    def __init__(self, matroid, weights, sample_prob):
        super().__init__(matroid, weights)
        
        # Sample S
        S = []
        self.P = set()
        for i in range(self.matroid.size()):
            if random.random() < sample_prob: 
                S.append((i, self.weights[i]))
            else: self.P.add(i)

        self.P = frozenset(self.P)
        S = dict(S)
        print(f'{S = }')
        self.X = self.matroid.max_weight_independent(X=set(S.keys()), weights=S) # max-weight basis of S
        self.X = sorted(list(self.X), key=lambda i: self.weights[i], reverse=True)

        self.current_iteration = 0
        self.current_span = frozenset()

    def run_trial(self):
        total = 0
        remaining = set(self.P) # which elements haven't been seen yet
        current_span = frozenset()
        A = frozenset() # store the answer

        for i, basis_elem in enumerate(self.X):
            next_span = self.matroid.closure(self.X[:i + 1]).intersection(self.P)
            candidates = list(next_span.difference(current_span))
            random.shuffle(candidates)
            
            print(f'{candidates = }')
            for y in candidates:
                remaining.remove(y)
                if self.weights[y] > self.weights[basis_elem]:
                    if self.matroid.is_independent(A.union(frozenset([y]))): 
                        A = A.union(frozenset([y]))
                        total += self.weights[y]
                        print(f'{y = }')
            current_span = next_span

        remaining = list(remaining)
        print(f'{remaining = }')
        random.shuffle(remaining)
        for y in remaining:
            if self.matroid.is_independent(A.union(frozenset([y]))): 
                A = A.union(frozenset([y]))
                total += self.weights[y]

        return total, A

In [37]:
# algo is assumed to be a FreeOrderMatroidAlgo. Returns averaged payoff over all trials
# weights will be a list of values of each item
def run_trials(num_trials, matroid, weights):
    sum = 0
    total_counts = defaultdict(lambda: 0)
    for _ in range(num_trials):
        algo = Conjecture(matroid, weights, 0.5)
        weight, included = algo.run_trial()
        sum += weight
        print(f'{included = }')
        for element in included: total_counts[element] += 1

    for key in total_counts: total_counts[key] = float(total_counts[key]) / num_trials
    return float(sum / num_trials), total_counts

In [6]:
random.seed(192)
#n = 5; rank = 3; numBases = 3
# M = random_matroid(n, rank, numBases)
# print(M)

# print(M.find_max_weight_basis({2: 3, 1: 5, 4: 3, 6: 10, 3: 2}))
# print(M.span([2, 4]))

# vectors = ((1, 2, 3), (4, 5, 6), (7, 8, 9), (10, 11, 12), (-4, 3, 2), (-5, -7, -8))
# n = len(vectors)
# M = VectorMatroid(vectors)
n = 10
M = knuth_random_matroid(n)
weights = [random.randint(0, 10) for i in range(n)]
print(weights)
val, freqs = run_trials(1000, M, weights)
X = set(range(n))

print(freqs)
opt = M.max_weight_independent(X=set(range(n)), weights = dict([(i, weight) for i, weight in enumerate(weights)]))
print(opt)

print(val)
tot = 0
for x in opt:
    tot += weights[x]

print(tot)

[10, 4, 5, 10, 4, 7, 6, 10, 3, 10]
defaultdict(<function run_trials.<locals>.<lambda> at 0x7f8215b773a0>, {9: 0.442, 1: 0.488, 0: 0.45, 8: 0.136, 3: 0.402, 7: 0.426, 2: 0.212, 5: 0.263, 6: 0.361, 4: 0.142})
frozenset({0, 1, 3, 7, 9})
25.195
44


In [ ]:
num_trials = 100
n = 10
results = []

for t in range(num_trials):
    start_time = time.time()
    M = knuth_random_matroid(n)
    end_time = time.time()
    # print("Time to create matroid of n = ", n, "is ", end_time - start_time)
    
    
    weights = [random.uniform(0, 1) for i in range(n)]
    
    
    start_time = time.time()
    val, freqs = run_trials(1000, M, weights)
    end_time = time.time()
    # print("Time to run 1000 trials is ", end_time - start_time)
    
    opt_indices = M.max_weight_independent(X=set(range(n)), weights = dict([(i, weight) for i, weight in enumerate(weights)]))
    opt_val = 0
    ratios = []
    for opt_index in opt_indices:
        opt_val += weights[opt_index]
        ratios.append(freqs[opt_index])
        #if (freqs[opt_index] < exp(-1)):
            #print("Less than 1/e when ", M, opt_index)
    
    results.append(min(ratios))

In [ ]:
print("Min:\t", round(min(results), 3))
print("Mean:\t", round(mean(results), 3))
print("Median:\t", round(median(results), 3))

In [75]:
def run_experiment(M, weights):
    start_time = time.time()
    num_trials = 5
    val, freqs = run_trials(num_trials, M, weights)
    end_time = time.time()
    print(f"Time to run {num_trials} trials is ", end_time - start_time)

    opt_indices = M.max_weight_independent(X=set(range(M.size())), weights = dict([(i, weight) for i, weight in enumerate(weights)]))
    opt_val = 0
    ratios = []
    
    for opt_index in opt_indices:
        ratios.append(freqs[opt_index])
        #if (freqs[opt_index] < exp(-1)):
            #print("Less than 1/e when ", M, opt_index)
    
    print(ratios)
    return min(ratios)

In [76]:
M, W = graph_counterexample(7)
print(run_experiment(M, W))

S = {1: 2, 3: 2}
candidates = []
candidates = []
remaining = [0, 2, 4, 5, 6]
included = frozenset({0, 2, 4, 6})
S = {2: 1, 4: 1, 5: 2, 6: 1}
candidates = []
candidates = []
candidates = []
candidates = [0, 3, 1]
y = 0
y = 3
y = 1
remaining = []
included = frozenset({0, 1, 3})
S = {1: 2, 3: 2, 4: 1, 6: 1}
candidates = []
candidates = []
candidates = [0, 2]
y = 0
candidates = [5]
y = 5
remaining = []
included = frozenset({0, 5})
S = {0: inf, 2: 1, 4: 1, 6: 1}
candidates = []
candidates = [1]
y = 1
candidates = [3]
y = 3
candidates = [5]
y = 5
remaining = []
included = frozenset({1, 3, 5})
S = {0: inf, 1: 2, 2: 1, 3: 2, 5: 2}
candidates = []
candidates = []
candidates = [4]
candidates = [6]
remaining = []
included = frozenset()
Time to run 5 trials is  0.006109952926635742
[0.6, 0.4, 0.4, 0.4]
0.4
